In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, FCC
import awkward as ak
import dask_awkward as dak
import numpy as np

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
fcc = FCC.get_schema()
events = NanoEventsFactory.from_root(
    '../coffea-fcc-analyses/data/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root:events',
    schemaclass=fcc,
    entry_stop=100,
    metadata={'dataset':'ZH'},
    delayed=True,
    uproot_options={"filter_name": lambda x : "PARAMETERS" not in x}
).events()

/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:226: UserWarning: Skipping _intMap as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:234: UserWarning: Skipping _intMap.first as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:234: UserWarning: Skipping _intMap.second as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:226: UserWarning: Skipping _floatMap as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:234: UserWarning: Skipping _floatMap.first as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:234: UserWarning: Skipping _floatMap.second as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:226: Us

In [3]:
events.ReconstructedParticles.fields

['E',
 'Electronidx0_indexGlobal',
 'MCRecoAssociationsidx0_indexGlobal',
 'Muonidx0_indexGlobal',
 'charge',
 'clusters',
 'covMatrix_10_',
 'goodnessOfPID',
 'mass',
 'particleIDs',
 'particles',
 'px',
 'py',
 'pz',
 'referencePoint',
 'tracks',
 'type']

#### Note the extra fields `Electronidx0_indexGlobal`, `MCRecoAssociationsidx0_indexGlobal`, and `Muonidx0_indexGlobal`.
#### They are the global indices to cross references for this collection

In [4]:
events.ReconstructedParticles.Muonidx0_indexGlobal.compute()

<Array [[-1, -1, -1, -1, ..., -1, -1, -1], ...] type='100000 * var * int64[...'>

#### -1 corresponds to no match (padding the array with -1 is necessary to ensure shape compatability)

In [5]:
print("Shape of ReconstructedParticles collection")
print(ak.num(events.ReconstructedParticles, axis =1).compute())
reco_cut = events.ReconstructedParticles.pt > 40.0
reco = events.ReconstructedParticles[reco_cut]
print("\nShape of ReconstructedParticles collection after pt > 40 cut")
print(ak.num(reco, axis =1).compute())

Shape of ReconstructedParticles collection
[60, 79, 55, 72, 50, 69, 56, 80, 62, ..., 106, 74, 52, 66, 37, 55, 78, 69, 68]

Shape of ReconstructedParticles collection after pt > 40 cut
[1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, ..., 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [6]:
muons = reco.match_muons
print("Shape of Matched Muons after pt > 40 cut")
print(ak.num(muons, axis=1).compute())

Shape of Matched Muons after pt > 40 cut
[1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, ..., 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [10]:
print("Shape of the Matched Generator(MC) particle from Matched Muons after pt > 40 cut")
mc_muons = muons.match_gen
print(ak.num(mc_muons, axis=1).compute())

Shape of the Matched Generator(MC) particle from Matched Muons after pt > 40 cut
[1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, ..., 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [11]:
mc_muons.PDG.compute()

<Array [[13], [13], [...], ..., [-2212], [13]] type='100000 * var * ?int32[...'>

In [20]:
from particle import Particle

In [22]:
Particle.from_pdgid(13)

<Particle: name="mu-", pdgid=13, mass=105.6583755 ± 0.0000023 MeV>

In [23]:
Particle.from_pdgid(211)

<Particle: name="pi+", pdgid=211, mass=139.57039 ± 0.00018 MeV>

In [29]:
Particle.from_pdgid(2212)

<Particle: name="p", pdgid=2212, mass=938.2720882 ± 0.0000003 MeV>